In [1]:
import asyncio
import re
import nltk
import requests
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio

# Apply nest_asyncio for Jupyter Notebook compatibility
nest_asyncio.apply()

# Download necessary NLTK data (run once)
nltk.download("punkt")

# Set your API keys (Replace with actual keys)
TOKEN = ""
HUGGINGFACE_API_KEY = ""

#Preprocessing Function
def preprocess_text(text):
    """Cleans and tokenizes user input text."""
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = text.lower().strip()
    words = word_tokenize(text)  # Tokenize text
    return " ".join(words)

#Sentiment Analysis Function
def analyze_sentiment(text):
    """Analyzes sentiment of the text and returns a label."""
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity  # Score between -1 and 1

    if sentiment_score > 0:
        return "positive 😊"
    elif sentiment_score < 0:
        return "negative 😔"
    else:
        return "neutral 😐"

#AI-Generated Response using Hugging Face (Facebook BlenderBot)
def generate_ai_response(user_text):
    """Generates AI-driven response using Hugging Face's Inference API."""
    API_URL = "https://api-inference.huggingface.co/models/facebook/blenderbot-400M-distill"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}
    payload = {"inputs": user_text}

    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()

        # Handle list response
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
            return result[0].get("generated_text", "Sorry, I couldn't generate a response.")
        elif isinstance(result, dict) and "generated_text" in result:
            return result["generated_text"]
        else:
            return "Sorry, I couldn't generate a response."
    
    return f"API Error: {response.status_code} - {response.text}"

#Telegram Bot Handlers
async def start(update: Update, context: CallbackContext):
    """Starts the bot with a welcome message."""
    await update.message.reply_text("Hello! How are you feeling today? 😊")

async def handle_message(update: Update, context: CallbackContext):
    """Handles user messages, processes text, analyzes sentiment, and generates AI responses."""
    user_text = update.message.text.lower().strip()

    if user_text in ["hi", "hello", "hey"]:
        await update.message.reply_text("Welcome! 😊 How are you feeling today?")
        return  # Exit here to wait for the user's response

    # Preprocess text
    clean_text = preprocess_text(user_text)

    # Analyze sentiment
    sentiment = analyze_sentiment(clean_text)

    # Generate AI response
    ai_response = generate_ai_response(clean_text)

    # Send response to user
    response = f"Your sentiment is: {sentiment}\n\n{ai_response}"
    await update.message.reply_text(response)

#Main Function to Run the Bot
async def main():
    """Runs the Telegram bot."""
    app = Application.builder().token(TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot is running...")
    await app.run_polling()

#Run the bot
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(main())  # ✅ Works in running event loops
    else:
        loop.run_until_complete(main())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91938\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Bot is running...
